In [1]:
from fuvtds_base_class import FUVTDSBase
from fuvtds_base_class import FUVTDSMonitor

In [2]:
x = FUVTDSBase(PIDs='fuvtds_analysis_list.dat')

/Users/jhernandez/Desktop/fuvtds/monitor/fuvtds_base_class.py:1098: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  programs_df = pd.read_csv(PIDs, delim_whitespace=True)


  0%|          | 0/1766 [00:00<?, ?it/s]

1533 FUVA
+++ Scaling LP6 to LP4 using data from datasets: /grp/hst/cos2/cosmo/16830/ler156gwq_x1d.fits.gz /grp/hst/cos2/cosmo/16830/ler156giq_x1d.fits.gz
1533 FUVB
+++ Scaling LP6 to LP4 using data from datasets: /grp/hst/cos2/cosmo/16830/ler15aknq_x1d.fits.gz /grp/hst/cos2/cosmo/16830/ler15al8q_x1d.fits.gz
1577 FUVA
+++ Scaling LP6 to LP4 using data from datasets: /grp/hst/cos2/cosmo/16830/ler156gyq_x1d.fits.gz /grp/hst/cos2/cosmo/16830/ler156gmq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv23mtq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv23mtq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv23mtq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /gr

In [4]:
df = FUVTDSMonitor(x)

URLError: <urlopen error [Errno 61] Connection refused>

In [ ]:
x.breakpoints

array([2010.2 , 2011.2 , 2011.75, 2012.  , 2012.8 , 2013.8 , 2015.5 ,
       2019.  , 2020.6 , 2022.  , 2023.2 ])

: 

: 

: 

In [ ]:
df.plot_solar_flux()

: 

: 

: 

: 

: 

: 